In [23]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
%matplotlib inline

# Load

In [24]:
df_train = pd.read_csv('../data/train_standard.csv', parse_dates=['date'], dtype={'fullVisitorId': 'str'})
df_test = pd.read_csv('../data/test_clean.csv', parse_dates=['date'], dtype={'fullVisitorId': 'str'})
with open('../models/classifier_1.pkl', 'rb') as pkl:
    classifier = pickle.load(pkl)
with open('../models/regressor_1.pkl', 'rb') as pkl:
    regressor = pickle.load(pkl)
with open('../models/scale_class.pkl', 'rb') as pkl:
    scaler = pickle.load(pkl)

In [29]:
# Separate
train_cols = [col for col in df_train.columns[2:] if col != 'totals.transactionRevenue']
test_cols = [col for col in df_test.columns[2:] if col != 'totals.transactionRevenue']
train = df_train[train_cols]
test = df_test[test_cols]

# Standardize
test = pd.DataFrame(data=scaler.transform(test), columns=test_cols)

# Predict Class
test['classifier_predict_probas'] = classifier.predict(test.values)

# Standardize (again)
# This has not been done yet, but need to do on the next run

# Predict Spending
reg_idxs = test[test['classifier_predict_probas'] > 0.01].index
reg_preds = regressor.predict(test.iloc[reg_idxs])
y_pred = pd.Series(np.zeros(len(test)))
for idx, pred in zip(reg_idxs, reg_preds):
    y_pred.set_value(idx, pred)
    
# Submission DF
ids = df_test['fullVisitorId']
submission = pd.DataFrame(data={'fullVisitorId': ids, 'PredictedLogRevenue': y_pred})

/Users/blairthurman/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/blairthurman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [33]:
# submission.to_csv('../data/submission_1.csv', index=False)

In [36]:
df_test.shape

(804684, 14)